# Bedrock Agent Memory: Episode機能の有無による比較検証

このNotebookでは、Amazon Bedrock AgentsのMemory機能において、**「Episode（セッション要約）機能」**が有効な場合と無効な場合で、記憶のされ方や思い出され方にどのような違いが出るかを検証します。

## 検証シナリオ: 「旅行の計画」
ユーザーがエージェントと2回に分けて会話を行います。

1.  **セッション1 (Episode 1):** 北海道旅行の計画（冬に行きたい、カニが食べたい）
2.  **セッション2 (Episode 2):** 沖縄旅行の計画（夏に行きたい、ダイビングしたい）

この2つのセッションを行った後、エージェントが記憶をどのように保持しているかを確認します。

## 構成
1.  **Agent A (Episode OFF):** 従来のMemory機能のみ（事実の記憶のみ）。
2.  **Agent B (Episode ON):** Session Summary（Episode）機能を有効化。

---

In [ ]:
# 必要なライブラリのインポート
import boto3
import json
import time
import uuid
import logging
from botocore.exceptions import ClientError

# ログ設定
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Boto3クライアントの初期化
region = 'us-east-1'  # Bedrock Agentが利用可能なリージョンを指定
bedrock_agent = boto3.client('bedrock-agent', region_name=region)
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name=region)
iam = boto3.client('iam')
sts = boto3.client('sts')

# アカウントIDの取得
account_id = sts.get_caller_identity()['Account']
print(f"Using Region: {region}")
print(f"Using Account ID: {account_id}")

## 1. 準備: IAMロールの作成
エージェントがBedrockのモデルを呼び出すための権限を持つIAMロールを作成します。

In [ ]:
role_name = f'BedrockAgentRole-EpisodeDemo-{str(uuid.uuid4())[:8]}'
assume_role_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

try:
    role = iam.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy)
    )
    logger.info(f"Created IAM Role: {role_name}")
    
    # モデル呼び出し権限の付与 (簡易的にBedrockFullAccessを付与)
    # 本番環境ではより制限されたポリシーを使用してください
    iam.attach_role_policy(
        RoleName=role_name,
        PolicyArn='arn:aws:iam::aws:policy/AmazonBedrockFullAccess'
    )
    
    role_arn = role['Role']['Arn']
    
    # IAMロールの伝播待ち
    time.sleep(10)
    
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']
        logger.info(f"Role already exists: {role_arn}")
    else:
        raise e

## 2. エージェント作成関数の定義
Episode機能（SessionSummary）のON/OFFを切り替えてエージェントを作成する関数を定義します。

* **`sessionSummaryConfiguration`**: ここがEpisode機能の肝です。`enabled=True`にすると、セッション終了時や一定時間経過後に、会話の要約が「エピソード」として生成・保存されます。

In [ ]:
def create_memory_agent(name, role_arn, enable_episode=False):
    """
    メモリ機能付きのエージェントを作成する関数
    enable_episode=Trueの場合、SessionSummary（Episode機能）を有効化する
    """
    
    # Memory設定の構築
    memory_config = {
        'storageDays': 30  # 記憶の保持期間
    }
    
    # ★ここがポイント: Episode機能のON/OFF
    if enable_episode:
        memory_config['sessionSummaryConfiguration'] = {
            'maxRecentSessions': 20, # 直近何件のセッション要約をコンテキストに含めるか
            'enabled': True
        }
    else:
        # 明示的に無効化（または設定しない）
        memory_config['sessionSummaryConfiguration'] = {
            'enabled': False
        }

    # エージェントの作成
    agent_name = f"{name}-{str(uuid.uuid4())[:8]}"
    response = bedrock_agent.create_agent(
        agentName=agent_name,
        agentResourceRoleArn=role_arn,
        foundationModel='anthropic.claude-3-sonnet-20240229-v1:0', # Claude 3 Sonnetを使用
        instruction="あなたは親切な旅行代理店のエージェントです。ユーザーの旅行計画を手伝い、好みを記憶してください。",
        memoryConfiguration=memory_config  # Memory設定を適用
    )
    
    agent_id = response['agent']['agentId']
    logger.info(f"Created Agent: {agent_name} (ID: {agent_id}, Episode: {enable_episode})")
    
    # エージェントの準備（Prepare）
    # エージェントを作成・変更した後はPrepareが必要
    time.sleep(5)
    bedrock_agent.prepare_agent(agentId=agent_id)
    
    # 準備完了まで少し待機（ポーリングするのが正式ですが今回はsleepで簡易化）
    time.sleep(10)
    
    # エイリアスの作成（Invokeにはエイリアスが必要）
    alias_response = bedrock_agent.create_agent_alias(
        agentId=agent_id,
        agentAliasName='v1',
        description='Version 1'
    )
    alias_id = alias_response['agentAlias']['agentAliasId']
    
    return agent_id, alias_id

## 3. エージェントの構築
比較用に2つのエージェントを作成します。
※ 作成には数分かかる場合があります。

In [ ]:
print("Creating Agent A (Episode OFF)...")
agent_a_id, agent_a_alias = create_memory_agent("Agent-No-Episode", role_arn, enable_episode=False)

print("Creating Agent B (Episode ON)...")
agent_b_id, agent_b_alias = create_memory_agent("Agent-With-Episode", role_arn, enable_episode=True)

print(f"\nCompleted.\nAgent A: {agent_a_id}\nAgent B: {agent_b_id}")

## 4. 会話実行ヘルパー関数の定義
エージェントと会話するための関数です。`enableMemory=True`を指定することで、会話内容がMemoryに保存されます。

In [ ]:
def chat_with_agent(agent_id, alias_id, session_id, memory_id, prompt):
    """
    エージェントと会話する関数
    """
    print(f"[User]: {prompt}")
    
    try:
        response = bedrock_agent_runtime.invoke_agent(
            agentId=agent_id,
            agentAliasId=alias_id,
            sessionId=session_id,
            memoryId=memory_id,      # Memory IDを指定（これで記憶が紐づく）
            enableTrace=False,
            enableMemory=True,       # Memory機能を有効化
            inputText=prompt
        )
        
        completion = ""
        for event in response.get('completion'):
            chunk = event['chunk']
            if chunk:
                completion += chunk['bytes'].decode('utf-8')
        
        print(f"[Agent]: {completion}\n")
        return completion
        
    except Exception as e:
        print(f"Error invoking agent: {e}")
        return None

## 5. 検証シナリオの実行

共通のMemory ID (`user_123`) を使用しますが、旅行ごとに**Session ID**を変えることで、別の「会話セッション（エピソード）」であることをエージェントに伝えます。

### シナリオA: 北海道旅行（セッション1）

In [ ]:
# 共通のMemory ID（同一ユーザーであることを表す）
memory_id = "user_mem_001"

# シナリオ用のセッションID
session_hokkaido = "session_hokkaido_001"
session_okinawa = "session_okinawa_001"

# 北海道旅行の会話スクリプト
scripts_hokkaido = [
    "こんにちは、冬休みの旅行計画を立てたいです。",
    "北海道に行ってみたいです。雪景色が見たいので。",
    "食べ物はやっぱりカニとラーメンが食べたいですね。札幌あたりがいいかな。",
    "ありがとう。予算は10万円くらいで考えておいて。"
]

print("--- Agent A (Episode OFF) : 北海道編 ---")
for text in scripts_hokkaido:
    chat_with_agent(agent_a_id, agent_a_alias, session_hokkaido, memory_id, text)
    
print("\n--- Agent B (Episode ON) : 北海道編 ---")
for text in scripts_hokkaido:
    chat_with_agent(agent_b_id, agent_b_alias, session_hokkaido, memory_id, text)

### シナリオB: 沖縄旅行（セッション2）
ここでセッションIDを切り替えます。現実では「数ヶ月後」などを想定しています。

In [ ]:
# 沖縄旅行の会話スクリプト
scripts_okinawa = [
    "こんにちは。今度は夏休みの旅行を考えたいです。",
    "沖縄でダイビングをしてみたいんです。",
    "食べ物はソーキそばとゴーヤチャンプルーがいいな。",
    "ホテルは海の見えるリゾートホテルでお願いします。"
]

print("--- Agent A (Episode OFF) : 沖縄編 ---")
for text in scripts_okinawa:
    chat_with_agent(agent_a_id, agent_a_alias, session_okinawa, memory_id, text)
    
print("\n--- Agent B (Episode ON) : 沖縄編 ---")
for text in scripts_okinawa:
    chat_with_agent(agent_b_id, agent_b_alias, session_okinawa, memory_id, text)

## 6. 結果確認: Memoryの中身を覗く

会話が終わったので、Agentがどのように記憶しているかを確認します。
`get_memory` APIを使用して、保存されている記憶を取得します。

* **Session Summary** が生成されるまでには、セッション終了後数分かかる場合がありますが、今回はAPI経由で強制的に確認を試みます。

### Agent A (Episode OFF) の記憶

In [ ]:
def show_memory_content(agent_id, agent_alias_id, memory_id, memory_type='ALL'):
    try:
        response = bedrock_agent_runtime.get_agent_memory(
            agentId=agent_id,
            agentAliasId=agent_alias_id,
            memoryId=memory_id,
            memoryType=memory_type
        )
        
        memories = response.get('memoryContents', [])
        if not memories:
            print("No memory found.")
            return

        for mem in memories:
            # セッションサマリー（Episode）がある場合
            if 'sessionSummary' in mem:
                summary = mem['sessionSummary']
                print(f"[Episode/Session ID]: {summary.get('sessionId')}")
                print(f"[Summary]: {summary.get('summaryText')}")
                print("-" * 30)
            
            # 通常のセッション履歴やコンテキスト情報はAPIの仕様により
            # get_agent_memoryでは主にSessionSummaryが返却される傾向にあります。
            # 生の会話履歴(SESSION_HISTORY)は取得できない場合があります。
            
    except Exception as e:
        print(f"Error getting memory: {e}")

print("=== Agent A (Episode OFF) Memory ===")
# Episode機能がOFFの場合、SessionSummaryは空のはずです
show_memory_content(agent_a_id, agent_a_alias, memory_id, 'SESSION_SUMMARY')

### Agent B (Episode ON) の記憶
こちらでは、北海道旅行と沖縄旅行がそれぞれ別の「Summary（要約）」として保存されていることが期待されます。
※ APIの反映に時間がかかる場合があるため、何も表示されない場合は少し待ってから再実行してください。

In [ ]:
print("=== Agent B (Episode ON) Memory ===")
# Episode機能がONの場合、各セッションごとの要約が表示されるはずです
print("Waiting for summaries to be generated (sleeping 30s)...")
time.sleep(30)
show_memory_content(agent_b_id, agent_b_alias, memory_id, 'SESSION_SUMMARY')

## 7. おまけ: 記憶に基づいた質問
最後に、エージェントに過去の記憶について質問してみます。

「以前、冬に計画していた旅行について教えて」と聞いた時、Episode機能があると**「北海道のことだな」**と文脈全体を引っ張りやすくなります。

In [ ]:
question = "以前、冬に計画していた旅行の内容を詳しく教えてくれる？"
new_session_id = "session_recall_001"

print(f"Q: {question}\n")

print("--- Agent A (Episode OFF) ---")
chat_with_agent(agent_a_id, agent_a_alias, new_session_id, memory_id, question)

print("\n--- Agent B (Episode ON) ---")
chat_with_agent(agent_b_id, agent_b_alias, new_session_id, memory_id, question)

## 8. クリーンアップ
作成したリソースを削除します。

In [ ]:
# エージェント削除関数
def delete_agent_resources(agent_id):
    try:
        # Agent Alias削除
        aliases = bedrock_agent.list_agent_aliases(agentId=agent_id)['agentAliasSummaries']
        for alias in aliases:
            bedrock_agent.delete_agent_alias(agentId=agent_id, agentAliasId=alias['agentAliasId'])
        
        # Agent削除
        bedrock_agent.delete_agent(agentId=agent_id)
        print(f"Deleted Agent: {agent_id}")
    except Exception as e:
        print(f"Error deleting agent {agent_id}: {e}")

# 削除実行
delete_agent_resources(agent_a_id)
delete_agent_resources(agent_b_id)

# IAMロール削除
try:
    iam.detach_role_policy(RoleName=role_name, PolicyArn='arn:aws:iam::aws:policy/AmazonBedrockFullAccess')
    iam.delete_role(RoleName=role_name)
    print(f"Deleted Role: {role_name}")
except Exception as e:
    print(f"Error deleting role: {e}")